In [4]:
import numpy as np
import pandas as pd
import openpyxl
from openpyxl import Workbook
from Levenshtein import distance as levenshtein_distance
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from rapidfuzz import fuzz
import functools
from collections import Counter
import nltk
from nltk.corpus import cess_esp
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import difflib
import re


In [ ]:
#Implements a variety of functions that preprocess in order to facilitate compararisons
#Turns all words into lowercase, removes whitespaces and lemantizes
#Also defines best match function to match the most accurate matches

nltk.download('cess_esp')
nltk.download('punkt')

#function to tag parts of speech
def pos_tagger(sentence):
    tokens = word_tokenize(sentence)
    tagged = nltk.pos_tag(tokens, lang='spa')
    return tagged

def autocomplete(word, word_list, n=1):
    matches = difflib.get_close_matches(word, word_list, n=n, cutoff=0.6)
    return matches[0] if matches else word


def preprocess_value_wordlist(value, spell, word_list):
    if pd.isna(value):
        return None
    value = str(value).lower().strip()
    words = value.split()
    autocompleted_words = [
        spell.correction(word) if word not in word_list else word
        for word in words
    ]
    autocompleted_words = [word if word is not None else '' for word in autocompleted_words]
    return ' '.join(autocompleted_words)

lemmatizer = WordNetLemmatizer()

def preprocess_value(value):
    if pd.isna(value):
        return None
    value = str(value).lower().strip()
    tokens = word_tokenize(value)
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    lemmatized_value = " ".join(lemmatized_tokens)
    return lemmatized_value

def tokenize(text):
    return text.lower().split() if text else []

memo = {}

def find_best_match(name1, name2, price1, price2, num1, num2, used_indices):
    best_score = -1
    best_index = -1
    best_match = None

    memo_key = (name1, tuple(name2), price1, tuple(price2), num1, tuple(num2))

    if memo_key in memo:
        return memo[memo_key]

    tokens1 = set(tokenize(name1))

    for idx, nombre2 in enumerate(name2):
        if idx in used_indices:
            continue

        tokens2 = set(tokenize(nombre2))

        word_matches = (len(tokens1.intersection(tokens2)))/len(tokens2)
        
        print(word_matches)

        fuzzy_score = fuzz.token_sort_ratio(name1, nombre2)

        # Combine word match count and fuzzy score
        combined_score = (word_matches * 100 + fuzzy_score)/2

        print(combined_score)

        if combined_score > best_score:
            best_score = combined_score
            best_index = idx
            best_match = (name1, price1, num1, nombre2, price2[idx], num2[idx] if num2 is not None else None, combined_score, fuzzy_score)

    memo[memo_key] = (best_match, best_index)
    
    return best_match, best_index

# Define memoization since its not defined for Rapid Fuzz
def memoize(func):
    cache = {}
    @functools.wraps(func)
    def memoized_func(*args):
        if args not in cache:
            cache[args] = func(*args)
        return cache[args]
    return memoized_func

@memoize
def fuzzy_match_score(s1, s2):
    return fuzz.token_sort_ratio(s1, s2)

def find_best_sku_match(id1, id2):
    for id2 in id2:
        if id1 in id2 or id2 in id1:
            return id2
        else:
            return None

def sku_match_score(id1, id2):
    if id1 == id2:
        return 100  
    elif id1 in id2 or id2 in id1:
        common_length = min(len(id1), len(id2))
        max_length = max(len(id1), len(id2))
        return 80 * (common_length / max_length)  
    else:
        return 0  

#Noramlized price difference returns a score for nearness in price

def normalize_price_diff(price1, price2):
    return 100 * (1 - min(abs(price1 - price2) / price1, 1))


def is_within_price_threshold(price1, price2, threshold_percentage):
    return np.abs(price1 - price2) / price1 <= threshold_percentage

def clean_id(value):
    try:
        value_str = str(value)
        if value_str.endswith('.0'):
            value_str = value_str[:-2]
        return value_str
    except ValueError:
        return value  

def extract_unique_words(column):
    unique_words = set()
    for description in column.dropna().unique():
        words = description.lower().strip().split()
        unique_words.update(words)
    return list(unique_words)

def apply_parallel(df, func, *args):
    with ThreadPoolExecutor() as executor:
        futures = list(tqdm(executor.map(lambda x: func(x, *args), df), total=len(df)))
    return futures

def remove_numeric_and_ec(text):
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\bec\b', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

This part of the code is responsible for defining functions to clean the data, as well as defining the matching logic which will be used later

In [ ]:
#Reads values from excel and csv depending on requirements can be changed. 

excel_file_1 = pd.read_excel("path to file 1")
excel_file_2 = pd.read_csv("path to file 2")

#Resets the index in order to make sure that empty excel cells dont cause a index error

excel_file_1 = excel_file_1[~excel_file_1["needed column"].isnull()] 
excel_file_1.reset_index(inplace=True)

database1 = excel_file_1[["name","price","id"]]
database1.columns = database1.columns.str.strip().str.replace(' ', '_')

database2 = excel_file_2[["name","avg_price","id", "avg_discount"]]

Reads from the specified files and I specifically reset the index because I was getting an error of having more cells than necessary because the excel files were formated in a bad manner

In [ ]:
tqdm.pandas()

# Preprocess data 1 
df_1 = database1.apply(lambda x: x.apply(preprocess_value) if x.name in ["name", "id"] else x)

df_1.dropna(subset=['price'], inplace=True)  

df_1['name'] = df_1['name'].apply(remove_numeric_and_ec)

df_1['price'] = pd.to_numeric(df_1['price'], errors='coerce') 

dictionary = set(df_1['name'].dropna().str.lower().str.split().sum())

print("base 1 done")

database2 = database2.rename(columns={"avg_price": "price", "avg_discount": 'discount',})

# Preprocess data 2 (database2) including discount column
df_2 = database2.apply(lambda x: x.progress_apply(preprocess_value) if x.name in ["name","id", "discount"] else x)

df_2['name'] = df_2['name'].progress_apply(remove_numeric_and_ec)

df_2['price'] = pd.to_numeric(df_2['price'], errors='coerce')

df_2['discount'] = pd.to_numeric(df_2['discount'], errors='coerce')

df_2.dropna(subset=['precio'], inplace=True)

print("base 2 done")

# Group by id and name in df_2_grouped

df_2_grouped = df_2(["id", "name"], as_index=False).agg(promedio_precio=('price', 'mean'))

df_2_grouped = df_2_grouped.rename(columns={'avg_price':'price'})

df_2['name'] = df_2_grouped['name'].progress_apply(lambda x: preprocess_value(x))

desc_prod_array = df_2_grouped['name'].to_numpy()

df_2['price'] = pd.to_numeric(df_2['price'], errors='coerce')  

# Apply the cleaning function to the id column in all df
df_1['id'] = df_1['id'].apply(clean_id)
df_2['id'] = df_2['id'].apply(clean_id)
df_2_grouped['id'] = df_2_grouped['id'].apply(clean_id)

# Check the results
print(df_2[['id']].head())
print(df_2_grouped[['id']].head())

print("df_1 info:")
df_1.info()

print("df_2 info:")
df_2.info()

print("df_2_agrupado info:")
df_2_grouped.info()

print("df_1 head:")
print(df_1.head())

print("df_2 head:")
print(df_2.head())

#Creates arrays in order to work easier and not have to reference which column in the data frames

name1_array = df_1['name'].values
price1_array = df_1['price'].values
numero1_array = df_1['id'].values

nombre2_array = df_2_grouped['name'].values
precio2_array = df_2_grouped['precio'].values
numero2_array = df_2_grouped['id'].values
discount2_array = df_2_grouped['discount'].values

Cleans the data and groups the values in the second document because the first document has the products we need to find and the second document is like recepits so we can find the matches

In [ ]:
counter = Counter(df_2_grouped['id'])

id_stats = df_2_grouped.groupby('id')['price'].agg(['max', 'min', 'mean', 'std']).reset_index()
id_stats.columns = ['id', 'Max Price', 'Min Price', 'Average Price', 'Std Deviation']

tokens1 = np.array([set(tokenize(text)) for text in df_1['name']])
tokens2 = np.array([set(tokenize(text)) for text in df_2_grouped['name']])

df_2_grouped['avg_discount'] = df_2.groupby('id')['discount'].transform('mean')

word_matches = np.zeros((len(df_1), len(df_2_grouped)), dtype=int)
fuzzy_scores = np.zeros((len(df_1), len(df_2_grouped)), dtype=int)
price_scores = np.zeros((len(df_1), len(df_2_grouped)), dtype=int)
sku_scores = np.zeros((len(df_1), len(df_2_grouped)), dtype=float)

for i in tqdm(range(len(df_1)), desc="Processing Rows", unit="row"):
    for j in range(len(df_2_grouped)):
        word_matches[i, j] = 100 * len(tokens1[i].intersection(tokens2[j])) / len(tokens1[i])
        fuzzy_scores[i, j] = fuzzy_match_score(df_1['name'].iloc[i], df_2_grouped['name'].iloc[j])
        price_scores[i, j] = normalize_price_diff(df_1['price'].iloc[i], df_2_grouped['price'].iloc[j])
        sku_scores[i, j] = sku_match_score(df_1['id'].iloc[i], df_2_grouped['id'].iloc[j])

counts the number of products in the grouped file and creates stats for the products, then gets the matches for each product as scores

In [6]:
# Define threshold and weight values
combined_score_threshold = 0
price_diff_threshold = 0.1
weight_word_matches = 0.45
weight_fuzzy_score = 0.50
weight_price_score = 0.05
weight_sku = 0.0

best_matches = []

used_indices_1_word_first = set()
used_indices_2_word_first = set()
used_indices_1_fuzzy_first = set()
used_indices_2_fuzzy_first = set()

#word first logic
for i, row1 in df_1.iterrows():
    sku1 = row1['id']
    best_match = None
    best_combined_score = -1  

    top_word_matches_indices = np.argsort(word_matches[i])[::-1][:10]  

    for best_index in top_word_matches_indices:
        if best_index not in used_indices_2_word_first:
            if is_within_price_threshold(df_1['price'][i], df_2_grouped['price'][best_index], price_diff_threshold):
                fuzzy_score = fuzzy_scores[i, best_index]
                combined_score = (word_matches[i, best_index] * weight_word_matches) + \
                                 (fuzzy_score * weight_fuzzy_score) + \
                                 (price_scores[i, best_index] * weight_price_score) + \
                                 (sku_scores[i, best_index] * weight_sku)

                if combined_score >= combined_score_threshold and combined_score > best_combined_score:
                    best_combined_score = combined_score
                    name1 = df_1['name'][i]
                    price1 = df_1['price'][i]
                    id1 = df_1['id'][i]
                    name2 = df_2_grouped['name'][best_index]
                    price2 = df_2_grouped['price'][best_index]
                    matched_id = df_2_grouped['id'][best_index]
                    repetitions = counter[matched_id]
                    avg_descuento = df_2_grouped['avg_discount'][best_index]

                    dif_price = np.abs(price1 - price2)

                    best_match = (
                        name1, price1, id1, name2, price2, matched_id,
                        combined_score, fuzzy_score, repetitions,
                        dif_price, avg_discount,
                        word_matches[i, best_index], price_scores[i, best_index]
                    )

    # Fuzzy First Logic
    top_fuzzy_matches_indices = np.argsort(fuzzy_scores[i])[::-1][:10] 

    for best_index in top_fuzzy_matches_indices:
        if best_index not in used_indices_2_fuzzy_first:
            if is_within_price_threshold(df_1['price'][i], df_2_grouped['price'][best_index], price_diff_threshold):
                word_score = word_matches[i, best_index]
                combined_score = (word_score * weight_word_matches) + \
                                 (fuzzy_scores[i, best_index] * weight_fuzzy_score) + \
                                 (price_scores[i, best_index] * weight_price_score) + \
                                 (sku_scores[i, best_index] * weight_sku)

                if combined_score >= combined_score_threshold and combined_score > best_combined_score:
                    best_combined_score = combined_score
                    name1 = df_1['name'][i]
                    price1 = df_1['price'][i]
                    id1 = df_1['id'][i]
                    name2 = df_2_grouped['name'][best_index]
                    price2 = df_2_grouped['price'][best_index]
                    matched_id = df_2_grouped['sku'][best_index]
                    repetitions = counter[matched_id]
                    avg_discount = df_2_grouped['avg_discount'][best_index]

                    dif_price = np.abs(price1 - price2)

                    best_match = (
                        name1, price1, id1, name2, price2, matched_id,
                        combined_score, fuzzy_scores[i, best_index], repetitions,
                        dif_price, avg_discount,
                        word_score, price_scores[i, best_index]
                    )

    if best_match:
        best_matches.append(best_match)

        used_indices_1_word_first.add(i)
        used_indices_1_fuzzy_first.add(i)
        used_indices_2_word_first.add(np.where(df_2_grouped['id'] == best_match[5])[0][0])  
        used_indices_2_fuzzy_first.add(np.where(df_2_grouped['id'] == best_match[5])[0][0]) 

df_best_matches = pd.DataFrame(best_matches, columns=[
    'doc 1 name', 'doc 2 price', 'doc 1 id',
    'doc 2 name', 'doc 2 price', 'doc 2 id',
    'Match Score', 'Fuzzy Score', '# de Repetitions', 'Price difference', 'average discount',
    'Word Score', 'Price Score'
])

df_best_matches['id Match Score'] = weight_sku
df_best_matches['Discount Percentage'] = ((df_best_matches['doc 2 price'] - df_best_matches['doc 1 price']) / df_best_matches['doc 1 price']) * 100
df_best_matches['trustworthyness'] = np.select(
    [df_best_matches['Match Score'] >= 59, df_best_matches['Match Score'] < 59],
    [2, 3],
    default=np.nan
)

df_best_matches = df_best_matches[df_best_matches['Match Score'] >= combined_score_threshold]

# Output to Excel
df_best_matches.to_excel("Combined_Best_Matches.xlsx", index=False)


After calculating the scores for each parameter we are using to compare we give it a weight depending on the files and what we consider relevant to the data so its customizable on a case by case basis